# Part 0. Google Colab Set Up

In [ ]:
# Mount Google Drive
from google.colab import drive

ROOT = "/content/drive"     
print(ROOT)                 

drive.mount(ROOT, force_remount=True)


In [ ]:
% cd /content/drive/MyDrive/rl_forex

## Anaconda Set Up

In [ ]:
% env PYTHONPATH=

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.9.2-Linux-x86_64.sh	
MINICONDA_PREFIX=/usr/local
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash

conda install --channel defaults conda=4.9.2 python=3.7 --yes
conda config --add channels conda-forge
conda config --set channel_priority strict

In [ ]:
import sys
 
# Add conda packages to PATH
_ = (sys.path
        .append("/usr/local/envs/rl_fx/lib/python3.7/site-packages"))

# Remove Colab preinstalled libraries
sys.path.remove('/usr/local/lib/python3.7/dist-packages')

## Import Conda Environment from yml 

In [ ]:
%%bash

conda env create -f environment.yml

In [ ]:
! source activate rl_fx && pip install yfinance==0.1.55


# Part 1. Import packages

In [17]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot

import sys

In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

<a id='1.4'></a>
## 2.4. Create Folders

In [20]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [21]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [22]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

ticker_list is a list of stock tickers, in a single stock trading case, the list contains only 1 ticker

In [23]:
from finrl.preprocessing import data, preprocessors
data_df=data.load_ohlc_dataset("15min/EURUSD/01_19.csv")

<a id='3'></a>
# Part 4. Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* FinRL uses a class **FeatureEngineer** to preprocess the data
* Add **technical indicators**. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc.


<a id='3.1'></a>

## 4.1 Technical Indicators
* FinRL uses stockstats to calcualte technical indicators such as **Moving Average Convergence Divergence (MACD)**, **Relative Strength Index (RSI)**, **Average Directional Index (ADX)**, **Commodity Channel Index (CCI)** and other various indicators and stats.
* **stockstats**: supplies a wrapper StockDataFrame based on the **pandas.DataFrame** with inline stock statistics/indicators support.



In [24]:
param_map = {
                'sma_9': {'talib_name' : 'SMA', 'time_period' : 9}, #time_period in seoonds
                'ema_9': {'talib_name' : 'EMA', 'time_period' : 9}, #time_period in seoonds
                'sma_21' : {'talib_name' : 'SMA', 'time_period' : 21},
                'ema_21' : {'talib_name' : 'EMA', 'time_period' : 21},
                'bbands_9':{'talib_name':'BBANDS','time_period':9,'nbdevup':2.0,'nbdevdn':2.0},
                'bbands_12':{'talib_name':'BBANDS','time_period':12,'nbdevup':2.0,'nbdevdn':2.0},
                'macd_entry':{'talib_name':'MACD', 'fastperiod':12, 'slowperiod':26,'signalperiod':9},
                'macd_exit':{'talib_name':'MACD', 'fastperiod':19, 'slowperiod':39,'signalperiod':9},
                'stoch':{'talib_name':'STOCH', 'fastk_period':5, 'slowk_period':3, 'slowk_matype':0, 'slowd_period':3, 'slowd_matype':0},
                'rsi_14':{'talib_name':'RSI', 'time_period':14},
                'rsi_4':{'talib_name':'RSI','time_period':4},
                'mom_10':{'talib_name':'MOM', 'time_period':10},
                'stochrsi_14':{'talib_name':'STOCHRSI', 'time_period':14, 'fastk_period':5,'fastd_period':3, 'fastd_matype':0},
                'kama_30':{'talib_name':'KAMA', 'time_period':30},
                't3_5':{'talib_name':'T3', 'time_period':5, 'vfactor':0.7},
                'atr_14':{'talib_name':'ATR', 'time_period':14},
                'natr_14':{'talib_name':'NATR', 'time_period':14},
                'tsf_14':{'talib_name':'TSF', 'time_period':14},
}

<a id='3.2'></a>
## 4.2 Perform Feature Engineering

In [25]:
from finrl.preprocessing import data, preprocessors

data_df = preprocessors.FeatureEngineer(data_df,
                          tech_indicator_params_map = param_map,
                          use_technical_indicator=True,
                          user_defined_feature=False,
                          tech_indicator_list = list(param_map.keys())).preprocess_data()

Successfully added technical indicators


For now, let's further preprocess the data to be a simple OHLC format with date

In [26]:
data_df

,ask_open,ask_high,ask_low,ask_close,bid_open,bid_high,bid_low,bid_close,bid_vol,ask_vol,...,rsi_14,rsi_4,mom_10,stochrsi_14_fastk,stochrsi_14_fastd,kama_30,t3_5,atr_14,natr_14,tsf_14
2019-01-02 09:30:00+00:00,1.14376,1.14423,1.14224,1.14277,1.14371,1.14419,1.14221,1.14273,6834.279998,7673.429994,...,29.197477,29.197477,-0.006010,0.000000,11.391039,1.146491,1.144176,0.001037,0.090779,1.142559
2019-01-02 09:45:00+00:00,1.14277,1.14348,1.14252,1.14344,1.14273,1.14345,1.14248,1.14340,6642.649976,7295.879995,...,35.013471,35.013471,-0.004915,61.727640,21.845314,1.146445,1.143679,0.001032,0.090274,1.142184
2019-01-02 10:00:00+00:00,1.14343,1.14381,1.14323,1.14362,1.14341,1.14378,1.14320,1.14358,7931.719974,8385.659979,...,36.522093,36.522093,-0.004380,98.616639,53.448093,1.146416,1.143362,0.001000,0.087435,1.141998
2019-01-02 10:15:00+00:00,1.14360,1.14383,1.14332,1.14377,1.14358,1.14380,1.14328,1.14374,6392.189985,6582.279992,...,37.859837,37.859837,-0.003915,100.000000,86.781426,1.146391,1.143217,0.000965,0.084395,1.141943
2019-01-02 10:30:00+00:00,1.14376,1.14420,1.14357,1.14386,1.14373,1.14418,1.14354,1.14384,5044.389977,4999.279998,...,38.712348,38.712348,-0.002380,100.000000,99.538880,1.146370,1.143208,0.000942,0.082325,1.142110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-31 22:45:00+00:00,1.14454,1.14470,1.14442,1.14470,1.14443,1.14461,1.14432,1.14460,1045.169999,1130.869995,...,40.843827,40.843827,0.000480,92.135731,30.711910,1.145026,1.144657,0.000439,0.038339,1.144720
2019-01-31 23:00:00+00:00,1.14467,1.14477,1.14461,1.14462,1.14459,1.14474,1.14456,1.14456,1829.169996,886.079999,...,39.909453,39.909453,0.000080,66.814849,52.983527,1.144968,1.144645,0.000420,0.036663,1.144726
2019-01-31 23:15:00+00:00,1.14462,1.14468,1.14450,1.14453,1.14457,1.14462,1.14446,1.14447,1418.089997,1182.009997,...,38.487168,38.487168,0.000010,30.685117,63.211899,1.144920,1.144622,0.000402,0.035108,1.144690
2019-01-31 23:30:00+00:00,1.14453,1.14478,1.14452,1.14466,1.14447,1.14474,1.14447,1.14462,1466.799993,1415.159995,...,41.952648,41.952648,0.000090,100.000000,65.833322,1.144908,1.144608,0.000392,0.034250,1.144680


In [27]:
train = data_df
train.reset_index(inplace = True, col_fill={'index':'date'})
train.rename(columns={
    "index" : "date"
}, inplace=True)
train

,date,ask_open,ask_high,ask_low,ask_close,bid_open,bid_high,bid_low,bid_close,bid_vol,...,rsi_14,rsi_4,mom_10,stochrsi_14_fastk,stochrsi_14_fastd,kama_30,t3_5,atr_14,natr_14,tsf_14
0,2019-01-02 09:30:00+00:00,1.14376,1.14423,1.14224,1.14277,1.14371,1.14419,1.14221,1.14273,6834.279998,...,29.197477,29.197477,-0.006010,0.000000,11.391039,1.146491,1.144176,0.001037,0.090779,1.142559
1,2019-01-02 09:45:00+00:00,1.14277,1.14348,1.14252,1.14344,1.14273,1.14345,1.14248,1.14340,6642.649976,...,35.013471,35.013471,-0.004915,61.727640,21.845314,1.146445,1.143679,0.001032,0.090274,1.142184
2,2019-01-02 10:00:00+00:00,1.14343,1.14381,1.14323,1.14362,1.14341,1.14378,1.14320,1.14358,7931.719974,...,36.522093,36.522093,-0.004380,98.616639,53.448093,1.146416,1.143362,0.001000,0.087435,1.141998
3,2019-01-02 10:15:00+00:00,1.14360,1.14383,1.14332,1.14377,1.14358,1.14380,1.14328,1.14374,6392.189985,...,37.859837,37.859837,-0.003915,100.000000,86.781426,1.146391,1.143217,0.000965,0.084395,1.141943
4,2019-01-02 10:30:00+00:00,1.14376,1.14420,1.14357,1.14386,1.14373,1.14418,1.14354,1.14384,5044.389977,...,38.712348,38.712348,-0.002380,100.000000,99.538880,1.146370,1.143208,0.000942,0.082325,1.142110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,2019-01-31 22:45:00+00:00,1.14454,1.14470,1.14442,1.14470,1.14443,1.14461,1.14432,1.14460,1045.169999,...,40.843827,40.843827,0.000480,92.135731,30.711910,1.145026,1.144657,0.000439,0.038339,1.144720
2838,2019-01-31 23:00:00+00:00,1.14467,1.14477,1.14461,1.14462,1.14459,1.14474,1.14456,1.14456,1829.169996,...,39.909453,39.909453,0.000080,66.814849,52.983527,1.144968,1.144645,0.000420,0.036663,1.144726
2839,2019-01-31 23:15:00+00:00,1.14462,1.14468,1.14450,1.14453,1.14457,1.14462,1.14446,1.14447,1418.089997,...,38.487168,38.487168,0.000010,30.685117,63.211899,1.144920,1.144622,0.000402,0.035108,1.144690
2840,2019-01-31 23:30:00+00:00,1.14453,1.14478,1.14452,1.14466,1.14447,1.14474,1.14447,1.14462,1466.799993,...,41.952648,41.952648,0.000090,100.000000,65.833322,1.144908,1.144608,0.000392,0.034250,1.144680


<a id='4'></a>
# Part 5. Build Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

<a id='4.1'></a>
## 5.1 Training & Trade data split
* Training: 2009-01-01 to 2018-12-31
* Trade: 2019-01-01 to 2020-09-30

<a id='4.2'></a>
## 5.2 User-defined Environment: a simulation environment class 

<a id='4.3'></a>
## 5.3 Initialize Environment
* **stock dimension**: the number of unique stock tickers we use
* **hmax**: the maximum amount of shares to buy or sell
* **initial amount**: the amount of money we use to trade in the begining
* **transaction cost percentage**: a per share rate for every share trade
* **tech_indicator_list**: a list of technical indicator names (modified from config.py)

In [28]:
train.columns[-28:]

Index(['sma_9', 'ema_9', 'sma_21', 'ema_21', 'bbands_9_upperband',
       'bbands_9_middleband', 'bbands_9_lowerband', 'bbands_12_upperband',
       'bbands_12_middleband', 'bbands_12_lowerband', 'macd_entry_macd',
       'macd_entry_macdsignal', 'macd_entry_macdhist', 'macd_exit_macd',
       'macd_exit_macdsignal', 'macd_exit_macdhist', 'stoch_slowk',
       'stoch_slowd', 'rsi_14', 'rsi_4', 'mom_10', 'stochrsi_14_fastk',
       'stochrsi_14_fastd', 'kama_30', 't3_5', 'atr_14', 'natr_14', 'tsf_14'],
      dtype='object')

In [29]:
## we store the stockstats technical indicator column names in config.py
## check https://github.com/jealous/stockstats for different names
tech_indicator_list = list(train.columns[-28:])
tech_indicator_list

['sma_9',
 'ema_9',
 'sma_21',
 'ema_21',
 'bbands_9_upperband',
 'bbands_9_middleband',
 'bbands_9_lowerband',
 'bbands_12_upperband',
 'bbands_12_middleband',
 'bbands_12_lowerband',
 'macd_entry_macd',
 'macd_entry_macdsignal',
 'macd_entry_macdhist',
 'macd_exit_macd',
 'macd_exit_macdsignal',
 'macd_exit_macdhist',
 'stoch_slowk',
 'stoch_slowd',
 'rsi_14',
 'rsi_4',
 'mom_10',
 'stochrsi_14_fastk',
 'stochrsi_14_fastd',
 'kama_30',
 't3_5',
 'atr_14',
 'natr_14',
 'tsf_14']

In [30]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(tech_indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 31


In [31]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "buy_cost_pct": 0.0001, 
    "sell_cost_pct": 0.0001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)
print(e_train_gym.data)

AttributeError: 'StockTradingEnv' object has no attribute 'state'

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

### Bayesian Optimised Training

In [ ]:
%autoreload
from finrl.model.callback import CustomCallback
from finrl.model.models import DRLAgent
import matplotlib.pyplot as plt



def A2C_train(learning_rate_val, rms_prop_eps_val):

    agent = DRLAgent(env = env_train)
    
    num_episodes = 100
    total_timesteps = num_episodes * len(train)

    A2C_PARAMS = {"n_steps": 20, "ent_coef": 0.001, "learning_rate": learning_rate_val, 'rms_prop_eps': rms_prop_eps_val}
    model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)
    trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=total_timesteps)

    # episode_rewards = trained_a2c.env.envs[0].get_episode_rewards()
    plt.plot(episode_rewards)
    plt.show()

    max_reward = max(episode_rewards)
    return max_reward

In [ ]:
from bayes_opt import BayesianOptimization

pbounds = {'learning_rate_val': (0.00001, 0.002), 'rms_prop_eps_val': (1e-06, 2e-05)}

optimizer = BayesianOptimization(
    f=A2C_train,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(init_points=2, n_iter=3)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC



### Model 1: A2C

In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000+(i-10)*10000, "learning_rate": 0.0001+(j-10)/200000}
        model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)
        trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=30000)
        print('i:',i,'j:',j)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)

In [ ]:
agent = DRLAgent(env = env_train)

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005+(i-10)/1000,
    "learning_rate": 0.0001+(j-10)/200000,
    "batch_size": 128,
}
        
        model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)
        trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)
        print('i:',i,'j:',j)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000, 
              "learning_rate": 0.0003}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000, 
              "learning_rate": 0.0003}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000+(i-10)*100000, 
              "learning_rate": 0.0003+(j-10)/20000}

        model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)
        trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)
        print('i:',i,'j:',j)

### Model 4: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.00003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.00003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

for i in range(20):
    for j in range(20):
        for k in range(20):
            agent = DRLAgent(env = env_train)
            SAC_PARAMS = {
        "batch_size": 128,
        "buffer_size": 100000+(i-10)*10000,
        "learning_rate": 0.00003+(j-10)/200000,
        "learning_starts": 100,
        "ent_coef": "auto_0.1",
    }
            model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)
            trained_sac = agent.train_model(model=model_sac, 
                                tb_log_name='sac',
                                total_timesteps=30000)
            print('i:',i,'j:',j,'k:',k)

### Trading
* we use the environment class we initialized at 5.3 to create a stock trading environment
* Assume that we have $100,000 initial capital at 2019-01-01. 
* We use the trained model of PPO to trade AAPL.

In [ ]:
## make a prediction and get the account value change
trade = data.load_ohlc_dataset("chicago_pmi/EURUSD/ohlc/EURUSD_Chicago_Pmi_2018-02-28.csv")

trade = preprocessors.FeatureEngineer(trade,
                          tech_indicator_params_map = param_map,
                          use_technical_indicator=True,
                          user_defined_feature=False,
                          tech_indicator_list = ["sma_9", "ema_9", "sma_21", "ema_21"]).preprocess_data()
trade = data_df['ovr'].iloc[29:]
trade.reset_index(inplace=True, col_fill='date')
trade.rename(columns={
    "time" : "date"
}, inplace=True)



e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                                           test_data = trade,
                                           test_env = env_trade,
                                           test_obs = obs_trade)

<a id='6'></a>
# Part 7: Backtesting Performance
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
%autoreload

In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = BackTestStats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to AAPL itself buy-and-hold===========")
%matplotlib inline
BackTestPlot(account_value=df_account_value, 
             baseline_ticker = 'AAPL',
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')

<a id='6.3'></a>
## 7.3 Baseline Stats

In [ ]:
print("==============Get Baseline Stats===========")
baesline_perf_stats=BaselineStats('AAPL')

In [ ]:
print("==============Get Baseline Stats===========")
baesline_perf_stats=BaselineStats('^GSPC')

<a id='6.4'></a>
## 7.4 Compare to Stock Market Index

In [ ]:
print("==============Compare to S&P 500===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
BackTestPlot(df_account_value, baseline_ticker = '^GSPC')